# Jet Extension


The purpose of this notebook is to explore the leading EOFs of the 850 hPa circulation, expressed through the streamfunction $\psi$.
This should provide insight as to the behavior of the SALLJ and hopefully may help us understand why the 2015-16 rainfall over the Lower Paraguay River Basin was so intense.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from eofs.xarray import Eof
from scipy.stats import spearmanr, pearsonr
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
import cartopy.feature
from paraguayfloodspy.xrutil import *
import paraguayfloodspy.visualize as viz
from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

## Setup

Load the parameters of the data

In [ ]:
%run ../config/RioParaguay.mk
%run ../config/WeatherTypes.mk
%run ../config/Time.mk
print(RPEAST, RPWEST, RPSOUTH, RPNORTH)
print(WTEAST, WTWEST, WTSOUTH, WTNORTH)
print(SYEAR, EYEAR)

In [ ]:
savefigs = True

Read in the raw data.
This data is the 850 hPa streamflow function $\psi$ which has been filtered to take the anomalies against the monthly 1980-2010 climatologies.

In [ ]:
psi = SelectData(xr.open_dataset('../data/derived/psi_850.nc'),
                    extent=[WTWEST, WTEAST, WTSOUTH, WTNORTH], y_low_high=True)
prcp = xr.open_dataset('../data/derived/precip.nc')

## EOFs

The next step of our analysis is to take the leading EOFs of the streamfunction.
We'll take out seasonality.

In [ ]:
solver = Eof(psi['anomaly'], center=True)

Let's look at the variance explained.
Note that the python indexing of the EOFs begins at zero but most people are used to seeing them begin at 1.

In [ ]:
%run ../config/WeatherTypes.mk
var_xpl = solver.varianceFraction(neigs=8).values
neof = np.min(np.where(np.cumsum(var_xpl) >= WT_PROP)) + 1
print(neof)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.xlabel("EOF Index")
plt.ylabel("Total Variance Explained")
plt.plot(np.arange(var_xpl.size) + 1, var_xpl)
plt.scatter(np.arange(var_xpl.size) + 1, var_xpl)
plt.title("EOF Variance")
plt.axvline(neof, color = 'black')
plt.grid(True)
plt.subplot(1,2,2)
plt.xlabel("EOF Index")
plt.ylabel("Cumulative Variance Explained")
plt.title("Cumulative Variance")
plt.plot(np.arange(var_xpl.size) + 1, np.cumsum(var_xpl))
plt.scatter(np.arange(var_xpl.size) + 1, np.cumsum(var_xpl))
plt.grid(True)
plt.axvline(neof, color = 'black')
if savefigs:
    plt.savefig("../figs/PSI_Var_Explained.pdf", bbox_inches='tight')

In [ ]:
loading = solver.eofs(neofs=neof)
loading['mode'] += 1

Let's also visualize the loadings

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, subplot_kw={'projection': ccrs.PlateCarree()}, 
                         figsize=(16, 3.5),sharex=True, sharey=True)
for i,m in enumerate(loading['mode'].values):
    ax = viz.GetRowCol(i, axes)
    ax.set_title("EOF {}".format(m))
    sub = loading.sel(mode = m)
    X, Y = np.meshgrid(sub.lon, sub.lat)
    C = ax.contourf(X, Y, sub, transform = ccrs.PlateCarree(), 
                     cmap = "PuOr", levels=np.linspace(-0.3, 0.3, 13))
fig.tight_layout()
fig.subplots_adjust(right=0.93)
cbar_ax = fig.add_axes([0.97, 0.05, 0.0125, 0.8])    
cb = plt.colorbar(C, cax=cbar_ax)
cb.set_label("EOF Loading", rotation=270)
cb.ax.get_yaxis().labelpad = 15
viz.FormatAxes(axes, coast=True, grid=False, border=True,
               ticks=[np.linspace(-180, 180, 73), np.linspace(-90, 90, 37)],
               extent=[WTWEST, WTEAST, WTSOUTH, WTNORTH])
if savefigs:
    plt.savefig("../figs/PSI_EOF_Loadings.pdf", bbox_inches='tight')    

Looking a this, we see that EOF 2 represents a SALLJ extension event, but that EOF 3 modulates whether it is a "Chaco Jet Event" 
or "No Chaco Jet Event"

In [ ]:
print("Chaco Jet Event")
Image(url= "http://www.eumetrain.org/satmanu/CMs/Sallj/media/images/CJEk.png", height=150)

In [ ]:
print("No Chaco Jet Event")
Image(url= "http://www.eumetrain.org/satmanu/CMs/Sallj/media/images/CM_SALLJ_NCJEk.png", height=150)

## PC Time Series

Now let's look at the time series of these two key PCs (2 and 3) and compare them to the rainfall field to see if it really matches our expectations.

In [ ]:
pcs = solver.pcs(npcs=neof, pcscaling=1)
pcs['mode'] += 1
pcs = pcs.to_pandas()

Now let's also create two time series of rainfall -- one over our Paragauy River Basin, and the other over the Chaco Jet Event region.

In [ ]:
prcp_rpy = xr.open_dataset("../data/derived/rainfall_rpy.nc")['raw'].to_pandas()
prcp_rpy = pd.DataFrame({'prcp_rpy': prcp_rpy})

In [ ]:
cje_rgn = {'lonmin': 299.25, 'lonmax': 302.75, 'latmin': -34.75, 'latmax': -30.75}
prcp_cje = prcp['raw'].sel(lon = slice(cje_rgn['lonmin'], cje_rgn['lonmax']), 
                    lat = slice(cje_rgn['latmin'], cje_rgn['latmax'])).mean(dim=['lon', 'lat'])
prcp_cje = prcp_cje.to_pandas()
prcp_cje = pd.DataFrame({'prcp_cje': prcp_cje})

In [ ]:
pc_rain = pcs.join(prcp_rpy).join(prcp_cje)
pc_rain = pc_rain.dropna()
pc_rain.head()

Correlation is obviously a limited tool, but let's see how the EOFs are correlated (daily time step) with the rainfall data

In [ ]:
print('Spearman rank correlation between URU and RPY Data: {:4f}'.format(spearmanr(pc_rain[['prcp_cje']], pc_rain[['prcp_rpy']])[0]))
print('Pearson correlation between URU and RPY Data: {:4f}'.format(pearsonr(pc_rain[['prcp_cje']], pc_rain[['prcp_rpy']])[0][0]))
for i in range(neof):
    print('Spearman rank correlation between EOF {} and RPY Data: {:3f}'.format(i+1, spearmanr(pc_rain[[i+1]], pc_rain[['prcp_rpy']])[0]))
    print('Pearson correlation between EOF {} and RPY Rainfall: {:3f}'.format(i+1, pearsonr(pc_rain[[i+1]], pc_rain[['prcp_rpy']])[0][0]))
    print('Spearman rank correlation between EOF {} and CJE Data: {:3f}'.format(i+1, spearmanr(pc_rain[[i+1]], pc_rain[['prcp_cje']])[0]))
    print('Pearson correlation between EOF {} and CJE Rainfall: {:3f}'.format(i+1, pearsonr(pc_rain[[i+1]], pc_rain[['prcp_cje']])[0][0]))

This correlation analysis supports our conjecture (based on looking at the EOF loadings) that EOF 2 and EOF 3 strongly modulate rainfall over the Paraguay River Basin.

## KNN

Let's focus on these two EOFs to get a bit better sense of how strongly they modulate intense rainfall.

In [ ]:
plt.figure(figsize=(11,8))
S = plt.scatter(pc_rain[[2]], pc_rain[[3]], s=5*pc_rain[['prcp_rpy']], c=pc_rain[['prcp_rpy']], alpha=0.4)
plt.xlabel("EOF 2")
plt.ylabel("EOF 3")
plt.title("Modulation of Rainfall by EOFs 2 and 3")
cb = plt.colorbar()
cb.set_label("Daily Rainfall [mm/day]")

It definitely looks like the top-left quadrant leads to higher probability of intense rainfall, but the plot is a bit crowded.
To get around this, let's do a nearest-neighbors regression -- this is essentially taking the expected rainfall given EOF 2 and EOF 3 -- at a grid of points.
Start by defining this grid and the fit to use:

In [ ]:
X = pc_rain[[2, 3]].values
neigh = KNeighborsRegressor(n_neighbors=100)
x = np.arange(-2.5, 2.5, .1)
x,y = np.meshgrid(x,x)
Xnew = np.array([np.hstack(x), np.hstack(y)]).transpose()
X1new = np.reshape(Xnew[:,0], x.shape)
X2new = np.reshape(Xnew[:,1], x.shape)
y = pc_rain[['prcp_rpy']].values
knnfit = neigh.fit(X,y)

Then create the plots

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
fig.subplots_adjust(right=0.95)
cbar_ax = fig.add_axes([0.97, 0.175, 0.02, 0.65])
# Left Plot
ax = axes[0]
y = pc_rain[['prcp_rpy']].values
knnfit = neigh.fit(X,y)
ynew = knnfit.predict(Xnew)
ynew = np.reshape(ynew, x.shape)
C = ax.pcolormesh(X1new, X2new, ynew, vmin=0, vmax=14)
ax.set_xlabel("EOF 2")
ax.set_ylabel("EOF 3")
ax.set_title("Lower Paraguay River Basin")
ax = axes[1]
y = pc_rain[['prcp_cje']].values
knnfit = neigh.fit(X,y)
ynew = knnfit.predict(Xnew)
ynew = np.reshape(ynew, x.shape)
C = ax.pcolormesh(X1new, X2new, ynew, vmin=0, vmax=14)
ax.set_xlabel("EOF 2")
ax.set_ylabel("EOF 3")
cb = plt.colorbar(C, cax=cbar_ax)
cb.set_label("Rainfall [mm/d]")
ax.set_title("Chaco Jet Extension Region")
if savefigs:
    plt.savefig("../figs/knn_rainfall_EOF.pdf", bbox_inches='tight')

So we can see that in this limited-dimension space, EOF 2 controls the potential for rainfall but EOF 3 controls whether it is a Chaco Jet Event or No Chaco Jet Event.
This is exactly what we thought above!
When EOF1 is positive (no jet), there is no rainfall in either region and when EOF1 is negative (jet) there is rainfall somewhere.
However, when EOF2 is negative the jet extends far to the South and the rainfall occurs over Uruguay; when EOF2 is positive the jet doesn't penetrate and the intense rainfall occurs in Paraguay.
Just to make sure we're not tricking ourselves, let's composite rainfall for four cases:
1. EOF1 > 1, EOF2 > 1; 
2. EOF1 < -1, EOF2 > 1;
3. EOF1 > 1, EOF2 < -1;
4. EOF1 < -1, EOF2 < -1.

In [ ]:
case_1 = np.logical_and(pc_rain[2] > 1, pc_rain[3] > 1)
case_1 = pc_rain.loc[case_1].index
case_2 = np.logical_and(pc_rain[2] < -1, pc_rain[3] > 1)
case_2 = pc_rain.loc[case_2].index
case_3 = np.logical_and(pc_rain[2] > 1, pc_rain[3] < -1)
case_3 = pc_rain.loc[case_3].index
case_4 = np.logical_and(pc_rain[2] < -1, pc_rain[3] < -1)
case_4 = pc_rain.loc[case_4].index

In [ ]:
fig, axes = viz.SetupAxes(nax=4, ncol=4, proj=ccrs.PlateCarree(), figsize=(16, 4.5))
fig.subplots_adjust(right=0.95)
cbar_ax = fig.add_axes([0.97, 0.1, 0.015, 0.8])
for i,case in enumerate([case_1, case_2, case_3, case_4]):
    selector = lambda ds: ds.sel(lon = slice(285, 315), lat = slice(-40, -5)).sel(time = case).mean(dim= 'time')
    ax = viz.GetRowCol(i, axes)
    ax.set_title("Case {}".format(i+1))
    sub = selector(prcp['raw'])
    Xp, Yp = np.meshgrid(sub.lon, sub.lat)
    sub = np.ma.masked_invalid(sub)
    C = ax.contourf(Xp, Yp, sub, transform = ccrs.PlateCarree(), 
                    levels=np.linspace(0,12,7), extend='max', cmap="Greens")
cb = plt.colorbar(C, cax=cbar_ax)
cb.set_label("Rainfall [mm/d]")
for ax in axes.flat:
    ax.coastlines()
    ax.set_extent((285, 315, -40, -5))
    ax.add_feature(cartopy.feature.BORDERS)

So looking at this, we can see that for Case 1 and Case 3 (no strong jet event) the Amazon basin experiences strong rainfall and SESA experiences very little rainfall.
However, during Case 2 and Case 4, during strong SALLJ occurrences, rainfall over the Amazon is reduced and rainfall below 20S is enhanced.
For case 2, this rainfall occurs over the Paraguay River Basin and for case 4 it occurs further South, over NE Argentina and Uruguay.

Just to check ourselves again, let's calculate the correlation of EOF 3 and the difference between the RPY rainfall box and CJE box.

In [ ]:
raindiff = pd.DataFrame({'prcp_diff': pc_rain['prcp_rpy'] - pc_rain['prcp_cje']}).join(pc_rain[[3]])
raindiff = raindiff.dropna()
rho, p = spearmanr(raindiff[['prcp_diff']], raindiff[[3]])
print("Spearman rank correlation between this difference and EOF 3 is {:3f}, p-value {}".format(rho, p))

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(raindiff[[3]].values, raindiff.prcp_diff.values)
plt.xlabel("EOF 3")
plt.ylabel("Rainfall Difference [mm/d]")

## S2S Controls

The results above suggest that, to a reasonable degree of approximation, differences in rainfall location (Paraguay River Basin or NE Argentina/Uruguay) are linked (in agreement with the literature) to different types of jet events.
Using the chosen domain, this corresponds to EOFs 2 and 3 of the NDJF streamfunction field (which is for all intents and purposes quite close to the geopotential height field).
To understand the 2015-16 event, we will explore: the time evolution of these two EOFs, their association with the observed rainfall, and seasonal (ENSO) and sub-seasonal (ENSO) control of their evolution.
To do that we'll have to resample the PC time series to monthly resolution.

In [ ]:
ensoraw = pd.read_csv("../data/accessed/monthly_indices.csv", index_col='time', parse_dates=True)
enso = ensoraw[['nino_34']]
enso['year'] = enso.index.year
enso['month'] = enso.index.month
enso = enso.loc[np.in1d(enso['month'], [11, 12, 1, 2])]
enso = enso.groupby(['year', 'month']).mean()

In [ ]:
ensopc = pcs[[2,3]].copy()
ensopc['year'] = ensopc.index.year
ensopc['month'] = ensopc.index.month
ensopc = ensopc.groupby(['year', 'month']).mean()
ensopc.head()
ensopc = ensopc.join(enso)

In [ ]:
sns.pairplot(ensopc, kind='reg')

So from this we can see that El Niño events are associated with enhanced monthly-mean SALLJ activity (EOF 2)  but that the association with EOF 3 is weak.

In [ ]:
spearmanr(ensopc[[2]], ensopc[['nino_34']])

In [ ]:
mjo = pd.read_csv("../data/accessed/daily_indices.csv",parse_dates=True, index_col='time')
mjo = mjo.join(pcs[[2,3]]).dropna()

In [ ]:
sns.pairplot(mjo[['RMM1', 'RMM2', 2, 3]], kind='reg', diag_kind="kde", plot_kws={'marker': '.'})

This suggests that at daily time scales, the strongest modulation is of RMM1 on EOF 3 (CJE/NCJE).

In [ ]:
spearmanr(mjo[[3]], mjo.RMM1)

In [ ]:
def read_indices(raw_url, col_name):
    df = pd.read_table(raw_url, delim_whitespace=True,index_col=None, skiprows=2, names=['time', '{}'.format(col_name)])
    df['time'] = np.int_(np.floor(df['time']))
    df['year'] = 1960 + df['time'] // 12
    df['month'] = 1 + df['time'] % 12
    df['day'] = 1
    df['time'] = pd.to_datetime(df[['year', 'month', 'day']])
    df.index = df['time']
    df = df[['{}'.format(col_name)]]
    return(df)

In [ ]:
url = 'http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/.EMC/.CMB/.GLOBAL/.Reyn_SmithOIv2/.monthly/.ssta/X/(35W)(10W)RANGEEDGES/Y/(10S)(40S)RANGEEDGES/Y+differences/[X+Y+]average/gridtable.tsv'
dipole = read_indices(url, col_name='sst')
#dipole = dipole.loc[dipole.index.month == 12]
dipole = dipole.resample('1D').ffill()
dipole = dipole.join(pcs.copy()[[2,3]]).dropna()

In [ ]:
sns.pairplot(dipole[[2, 'sst']], kind='reg', diag_kind="kde", plot_kws={'marker': '.'})

In [ ]:
sns.pairplot(dipole[[3, 'sst']], kind='reg', diag_kind="kde", plot_kws={'marker': '.'})

In [ ]:
print(spearmanr(dipole[[2]], dipole['sst']))
print(spearmanr(dipole[[3]], dipole['sst']))